In [2]:
!pip install git+https://github.com/suno-ai/bark.git
!pip install nltk scipy

[nltk_data] Downloading package punkt to /home/xiaolai/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import nltk  # we'll use this to split into sentences
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/xiaolai/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# if you're on MacOS, especially with M1/M2 CPUs, you'll need to set MPS to "true"
# os.environ["SUNO_ENABLE_MPS=true"] = True

from IPython.display import Audio
import numpy as np
from bark.generation import (
    generate_text_semantic,
    preload_models,
)
from bark.api import semantic_to_waveform
from bark import generate_audio, SAMPLE_RATE

preload_models()

print("Ready!")

Ready!


 # Long-Form Generation

In [9]:
script = """
The results cannot be talked away. But in many intellectual areas, notably so-called social science, there is neither a swift nor a certain authentication process for ideas. And the only ultimate validation is whether the ideas sound plausible to enough people or to the right people.
""".replace("\n", " ").strip()

sentences = nltk.sent_tokenize(script)
print('Splitted into sentences...\n')

for s in sentences:
    print(s)

print('\nDone!')

Splitted into sentences...

The results cannot be talked away.
But in many intellectual areas, notably so-called social science, there is neither a swift nor a certain authentication process for ideas.
And the only ultimate validation is whether the ideas sound plausible to enough people or to the right people.

Done!


In [10]:
GEN_TEMP = 0.9 # the greater the compact of the generated audio
SPEAKER = "v2/en_speaker_3"
# in male voices, 2, 3, 6 are nicer;
# only 9 is female voice.

silence = np.zeros(int(0.15 * SAMPLE_RATE))  # 0.25 is quarter second of silence

pieces = []
print('Generating audio of following sentences...')
for sentence in sentences:
    semantic_tokens = generate_text_semantic(
        sentence,
        history_prompt=SPEAKER,
        temp=GEN_TEMP,
        min_eos_p=0.05,  
        # this controls how likely the generation is to end
        # Somtimes Bark will hallucinate a little extra audio 
        # at the end of the prompt. We can solve this issue 
        # by lowering the threshold for bark to stop generating 
        # text. We use the min_eos_p kwarg in generate_text_semantic
    )
    print(sentence)

    audio_array = semantic_to_waveform(semantic_tokens, history_prompt=SPEAKER,)
    pieces += [audio_array, silence.copy()]
    

print('Generating wav...')

Audio(np.concatenate(pieces), rate=SAMPLE_RATE)

Generating audio of following sentences...


100%|██████████| 191/191 [00:01<00:00, 163.49it/s]


The results cannot be talked away.


100%|██████████| 673/673 [00:05<00:00, 115.32it/s]


But in many intellectual areas, notably so-called social science, there is neither a swift nor a certain authentication process for ideas.


100%|██████████| 430/430 [00:03<00:00, 139.85it/s]


And the only ultimate validation is whether the ideas sound plausible to enough people or to the right people.


100%|██████████| 22/22 [00:03<00:00,  5.90it/s]


Generating wav...


# Make a long-form Dialog

In [ ]:
# Step 1: Format a script and speaker lookup
GEN_TEMP = 0.9
SPEAKER = "v2/en_speaker_9"
silence = np.zeros(int(0.15 * SAMPLE_RATE))  # quarter second of silence

# Step 2: Generate the audio for every speaker turn
pieces = []
print('Generating audio of following sentences...')
for sentence in sentences:
    semantic_tokens = generate_text_semantic(
        sentence,
        history_prompt=SPEAKER,
        temp=GEN_TEMP,
        min_eos_p=0.05,  # this controls how likely the generation is to end
    )
    print(sentence)

    audio_array = semantic_to_waveform(semantic_tokens, history_prompt=SPEAKER,)
    pieces += [audio_array, silence.copy()]
    

print('Generating wav...')

# Step 3: Concatenate all of the audio and play it
Audio(np.concatenate(pieces), rate=SAMPLE_RATE)